In [1]:
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import utils
import funcs
import eda_plots
import new_mlfuncs
import xgboostfuncs
import dlfuncs

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
# import train data
train_filepath = 'data/df_merged.csv'
train_df, train_df_nulls = funcs.read_and_qa(train_filepath)

There are 41 columns and 59400 records in the dataframe.
Extracting count and percentages of nulls and non nulls


In [4]:
# import test data
test_filepath = 'data/test_values.csv'
test_df, test_df_nulls = funcs.read_and_qa(test_filepath)

There are 40 columns and 14850 records in the dataframe.
Extracting count and percentages of nulls and non nulls


In [5]:
# initial columns to drop
initial_to_drop = ['id', 'date_recorded', 'num_private', 'region', 'region_code', 'recorded_by', 'extraction_type_group', 'extraction_type_class',\
                  'quantity_group', 'source_type', 'waterpoint_type_group', 'permit', 'funder', 'installer', 'wpt_name', 'subvillage', 'lga', 'ward',\
                  'management_group', 'payment']
target_var = 'status_group'
num_cols = []
feats_to_exclude = []

In [6]:
train_red_df = new_mlfuncs.pre_ml_preprocessing(train_df, initial_to_drop, num_cols, target_var = 'status_group', missing_val_threshold = 40, num_cols_threshold = 0.9, low_var_threshold = 0.9)

Dropping: id
Dropping: date_recorded
Dropping: num_private
Dropping: region
Dropping: region_code
Dropping: recorded_by
Dropping: extraction_type_group
Dropping: extraction_type_class
Dropping: quantity_group
Dropping: source_type
Dropping: waterpoint_type_group
Dropping: permit
Dropping: funder
Dropping: installer
Dropping: wpt_name
Dropping: subvillage
Dropping: lga
Dropping: ward
Dropping: management_group
Dropping: payment
Dropping scheme_name due to missing values count above threshold: 0.4
There are no high correlated features. No column will be dropped!
Dropping low variance feature: public_meeting


In [7]:
test_red_df = new_mlfuncs.pre_ml_preprocessing(test_df, initial_to_drop, num_cols, target_var = None, missing_val_threshold = 40, num_cols_threshold = 0.9, low_var_threshold = 0.9)

Dropping: id
Dropping: date_recorded
Dropping: num_private
Dropping: region
Dropping: region_code
Dropping: recorded_by
Dropping: extraction_type_group
Dropping: extraction_type_class
Dropping: quantity_group
Dropping: source_type
Dropping: waterpoint_type_group
Dropping: permit
Dropping: funder
Dropping: installer
Dropping: wpt_name
Dropping: subvillage
Dropping: lga
Dropping: ward
Dropping: management_group
Dropping: payment
Dropping scheme_name due to missing values count above threshold: 0.4
There are no high correlated features. No column will be dropped!
Dropping low variance feature: public_meeting


In [8]:
train, test, le = new_mlfuncs.target_encode_categorical_features(train_red_df, test_red_df, target=target_var, alpha=10)

In [9]:
target_var = 'target_encoded'

In [11]:
acc_score, pipe, final_predictors = xgboostfuncs.xgboost_model(train, target_var, text_feats=None)

Creating pipeline for XGBoost Classifier.
Fitting training data to pipeline for XGBoost Classifier.
[02:32:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Leonard\anaconda3\envs\PythonAdv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Predicting test values for XGBoost Classifier.
Calculating accuracy score for XGBoost Classifier.
Accuracy Score for XGBoost Classifier: 0.7921619479864563


In [12]:
GridSearchCV_model_output = xgboostfuncs.best_classifier_hyperparameter(train, target_var)

hyperparameter tuning for the Best Classifier: XGBClassifier
Creating pipeline for XGBClassifier.
Fitting XGBClassifier Grid Model to training data.
[21:27:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Leonard\anaconda3\envs\PythonAdv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Users\Leonard\anaconda3\envs\PythonAdv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:27:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Leonard\anaconda3\envs\PythonAdv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:28:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

[02:11:22] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Leonard\anaconda3\envs\PythonAdv\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:14:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Best parameters after GridSearchCV:
 {'xgbclassifier__learning_rate': 0.05, 'xgbclassifier__max_depth': 6, 'xgbclassifier__n_estimators': 1000}
Best score after GridSearchCV:
 0.80


C:\Users\Leonard\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
model_path = 'models/new_xgb.pkl'

In [ ]:
new_mlfuncs.dump_estimator(GridSearchCV_model_output, model_path)

In [ ]:
# model_path = 'models/randomforest_1.pkl'
# df_to_predict_path = 'data/test_values.csv' 
submission_format_path = 'data/SubmissionFormat.csv'
# model_path = 'models/best_classifier_dump.pkl'
my_submission_path = 'data/new_xgb_classifier.csv'

In [ ]:
submission = new_mlfuncs.make_predictions(test, submission_format_path,  my_submission_path, model_path, final_predictors, le)